# SPECFEM Users Workshop -- Day 1 (Oct. 5, 2022)

## Forward Simulations in SPECFEM2D and 3D

This notebook is meant to walk Users through an introduction to `SPECFEM2D`, from a) navigating a SPECFEM2D working directory, to b) generating a mesh, to c) running forward simulations. We make note of important files and steps to take when running forward simulations in SPECFEM. Background material is included to understand the task at hand, and we also draw parallels with `SPECFEM3D`, illustrating how to mesh and run simulations on a simple 3D domain. At the end of the workshop, we will take a look at how we can use `SeisFlows` to automate SPECFEM2D for running forward simulations.

-----------

### 0) Relevant Information

>__NOTE:__ These instructions should be run from inside the Docker container, using Jupyter Lab. The Docker container should have the adjTomo toolkit installed (SeisFlows, Pyatoa, PySEP), as well as SPECFEM2D and SPECEFM3D compiled with MPI. 

**Relevant Links:** 
- !!! ADD DAY 1 SLIDES HERE !!!
- Workshop Material: https://github.com/adjtomo/adjdocs/tree/main/workshops/2022-10-05_specfem_users
- Today's Notebook: !!! ADD THIS !!!

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the `Run` button at the top, or by hitting `Shift + Enter`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. Where the number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands

## 1) Background !!! TODO !!!


Potential topics: 
- Seismic waveforms
- Numerical modeling
- Spectral element method
- Meshes


## 2) SPECFEM2D Tour

>__NOTE__: The `devel` branch of all SPECFEM versions contains the most up to date codebase

- GitHub Repository: https://github.com/geodynamics/specfem2d/tree/devel
- SPECFEM2D Manual: https://specfem2d.readthedocs.io/en/latest/

### a) Binary Executables in *bin/* directory

In this workshop container, we have already downloaded (git clone), configured (choosing compilers and compiler options) and compiled (make all) SPECFEM2D. The binary executable files are located in the `bin/` directory. Each of these executables performs a different function in the package.

The two most important executables we will be using today are `xmeshfem2D` and `xspecfem2D`. 
- `xmeshfem2D` is used to generate our numerical mesh, the skeleton of the domain upon which we run our numerical simulations. 
- `xspecfem2D` runs the spectral element solver, generating synthetic seismograms for a given source and set of stations.

In [ ]:
# Let's have a look at the executables
! ls /home/scoped/specfem2d/bin

### b) Metadata in *DATA/* directory

Data that the User will provide to SPECFEM should be stored in the *DATA/* directory. The most important files that we will concern ourselves with are the `Par_file`, `SOURCE` and `STATIONS` files.

-`Par_file`: The parameter file which allows the User to adjust parameters for a given simulation  
-`SOURCE`: Defines source characteristics (e.g., moment tensor, force). **NOTE**: SPECFEM2D and SPECFEM3D have a number of different types of available source files (e.g., SOURCE, FORCESOLUTION, CMTSOLUTION)  
-`STATIONS`: Defines station codes and locations (either Cartesian or geographic)

The following commands open these files for the SPECFEM2D example problem

In [ ]:
! ls /home/scoped/specfem2d/DATA

In [ ]:
! head -50 /home/scoped/specfem2d/DATA/Par_file

In [ ]:
! cat /home/scoped/specfem2d/DATA/SOURCE

In [ ]:
! cat /home/scoped/specfem2d/DATA/STATIONS

### c) Results stored in *OUTPUT_FILES/* directory

Any outputs generated by SPECFEM will be stored in the `OUTPUT_FILES/` directory. These include log and error messages, synthetic seismograms, figures, and output files of one executable that may be required by another.

>__NOTE__: SPECFEM also maintains a DATABASE directory (typically called `DATABASES_MPI/`) which is used to store large database files containing the entire GLL mesh and model, as well as intermediate files such as the saved forward wavefield. This directory may be the same as `OUTPUT_FILES/`, or may be it's own separate directory.

In [ ]:
# Currently empty because we have not run any executables
! ls /home/scoped/specfem2d/OUTPUT_FILES

## 3) Running the mesher `xmeshfem2D` 

The first thing we need to do when approaching numerical simulations is to generate our numerical mesh. There are multiple approaches to meshing, such as using external software such as Trellis. During this workshop we will use SPECFEM's internal meshing software, known as `Meshfem`.

### a) Mesh Parameters

In SPECFEM2D, mesh parameters are defined in the `Par_file`. There are various parameter options we can use to customize our mesher run.

>__NOTE:__ In SPECFEM3D, Meshfem files are defined separately to provide more control over a 3D domain. These files are typically stored in `specfem3d/DATA/meshfem3D_files`. We will look at these later in the notebook.